In [1]:
colab = False

import torch
import os 

if colab:
  print("PyTorch has version {}".format(torch.__version__))

  # Install torch geometric
  if 'IS_GRADESCOPE_ENV' not in os.environ:
    !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-geometric

  !git clone https://github.com/thibautvalour/Graph-Diffusion-Convolution.git
  %cd Graph-Diffusion-Convolution

import numpy as np
import networkx as nx
import pandas as pd
import math as math
from torch_geometric.datasets import Planetoid
from torch.nn.functional import nll_loss

from matrix_format import gdc_pagerank, gdc_heat, compute_Lsym, compute_Lrw
from models import GCN_Classifier
from utils import train, test

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

c:\Users\33646\Desktop\ENSAE\Structured data\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cpu


# 1 Cora

In [2]:
dataset = Planetoid(root='../cora', name='Cora')
data = dataset[0]

test_prop = 0.2
train_idx = torch.tensor(np.random.binomial(1, 1-test_prop, size=data.y.shape[0])).to(bool)
test_idx = torch.tensor(np.logical_not(train_idx)).to(bool).to(device)
train_idx = train_idx.to(device)

Gnx = nx.from_pandas_edgelist(pd.DataFrame(data['edge_index'].T,
                                           columns=['source', 'target']))
Gnx = Gnx.to_undirected()

# Exctract adjacency matrix
A = nx.adjacency_matrix(Gnx).toarray()
data = data.to(device)

C:\Users\33646\AppData\Local\Temp\ipykernel_10772\351346556.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_idx = torch.tensor(np.logical_not(train_idx)).to(bool).to(device)


### 1.1 Laplacian matrix

In [3]:
Lsym = compute_Lsym(A)
Lsym = torch.from_numpy(Lsym).float().to(device)

In [4]:
args = {
    'device': device,
    'hidden_layers': 1,
    'hidden_dim': 264,
    'dropout': 0.3,
    'lr': 3e-4,
    'epochs': 80,
    'trans_matrix': Lsym
}

In [5]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

Epoch: 01, Loss: 2.0701, Train: 24.87%, Test: 24.38%
Epoch: 02, Loss: 1.9488, Train: 29.63%, Test: 27.26%
Epoch: 03, Loss: 1.8533, Train: 34.80%, Test: 29.94%
Epoch: 04, Loss: 1.7562, Train: 39.69%, Test: 32.63%


KeyboardInterrupt: 

In [ ]:
Lrw = compute_Lrw(A)
Lrw = torch.from_numpy(Lrw).float().to(device)
args['trans_matrix'] = Lrw

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

### 1.2 Page rank

In [ ]:
pagerank = gdc_pagerank(A, 0.05, 1e-4)
pagerank = torch.from_numpy(pagerank).float().to(device)

args = {
    'device': device,
    'hidden_layers': 1,
    'hidden_dim': 264,
    'dropout': 0.3,
    'lr': 3e-4,
    'epochs': 80,
    'trans_matrix': pagerank
}

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

### 1.3 Heat

In [ ]:
heat = gdc_heat(A, 3, 10, 1e-4)
heat = torch.from_numpy(heat).float().to(device)

args['trans_matrix'] = heat

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

# Amazon

In [3]:
from torch_geometric.datasets import Amazon

dataset = Amazon(root='../cora', name='Computers')
data = dataset[0]

test_prop = 0.2
train_idx = torch.tensor(np.random.binomial(1, 1-test_prop, size=data.y.shape[0])).to(bool)
test_idx = torch.tensor(np.logical_not(train_idx)).to(bool).to(device)
train_idx = train_idx.to(device)

Gnx = nx.from_pandas_edgelist(pd.DataFrame(data['edge_index'].T,
                                           columns=['source', 'target']))
Gnx = Gnx.to_undirected()

# Exctract adjacency matrix
A = nx.adjacency_matrix(Gnx).toarray()
data = data.to(device)

C:\Users\33646\AppData\Local\Temp\ipykernel_10772\2585882802.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_idx = torch.tensor(np.logical_not(train_idx)).to(bool).to(device)


## 2.1 Laplacian

In [ ]:
Lsym = compute_Lsym(A)
Lsym = torch.from_numpy(Lsym).float().to(device)

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

## 2.2 page rank

In [ ]:
pagerank = gdc_pagerank(A, 0.05, 1e-4)
pagerank = torch.from_numpy(pagerank).float().to(device)
args['trans_matrix'] = pagerank

model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

## 2.3 Heat Kernel

In [ ]:
heat = gdc_heat(A, 3, 10, 1e-4)
heat = torch.from_numpy(heat).float().to(device)

args['trans_matrix'] = heat

model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')